Import all libraries


In [ ]:
import os
import zipfile
import requests
import nltk
import matplotlib.pyplot as plt
import fasttext as ft
import fasttext.util as ftutil
import numpy as np
import re
import pandas as pd
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.stem.snowball import SnowballStemmer
from tqdm import tqdm
from nltk.corpus.reader.wordnet import WordNetCorpusReader
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import normalize
from wnaffect import WNAffect
from emotion import Emotion
from googletrans import Translator
from collections import defaultdict

Implement download functions to download the data.


In [4]:
def download_and_extract(url, zip_path, extract_path):
    if os.path.exists(extract_path):
        print(f"File {extract_path} already exists.")
    else:
        print(f"Downloading and extracting the {zip_path} to {extract_path}.")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024
        with open(zip_path, "wb") as file, tqdm(
            desc=zip_path,
            total=total_size,
            unit="iB",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(block_size):
                file.write(data)
                bar.update(len(data))
        if not os.path.exists(extract_path):
            os.makedirs(extract_path)
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            for member in zip_ref.namelist():
                member_path = os.path.join(
                    extract_path,
                    os.path.relpath(
                        member, start=os.path.commonpath(zip_ref.namelist())
                    ),
                )
                if member.endswith("/"):
                    os.makedirs(member_path, exist_ok=True)
                else:
                    os.makedirs(os.path.dirname(member_path), exist_ok=True)
                    with open(member_path, "wb") as f:
                        f.write(zip_ref.read(member))
        os.remove(zip_path)
        print(f"Download and extraction of {extract_path} complete.")


def download(url, file_path):
    if os.path.exists(file_path):
        print(f"File {file_path} already exists.")
    else:
        print(f"Downloading the file {file_path}")
        response = requests.get(url, stream=True)
        total_size = int(response.headers.get("content-length", 0))
        block_size = 1024

        with open(file_path, "wb") as file, tqdm(
            desc=file_path,
            total=total_size,
            unit="iB",
            unit_scale=True,
            unit_divisor=1024,
        ) as bar:
            for data in response.iter_content(block_size):
                file.write(data)
                bar.update(len(data))
        print(f"Download of {file_path} complete.")

Download the Finnish Wikipedia 2017 dataset, Finnish Stopwords, Stemmer and FinnWordNet.


In [ ]:
# Wikipedia dataset
wiki_url = "https://www.kielipankki.fi/download/wikipedia-fi/wikipedia-fi-2017-src/wikipedia-fi-2017-src.zip"
wiki_zip_path = "wikipedia-fi-2017-src.zip"
wiki_extract_path = "data/wikipedia-fi-2017-src"
download_and_extract(wiki_url, wiki_zip_path, wiki_extract_path)

# Finnish Stopwords
stopword_url = "http://members.unine.ch/jacques.savoy/clef/finnishST.txt"
stopwords_path = "data/finnishST.txt"
download(stopword_url, stopwords_path)

# Finnish Stemmer
# stemmer_url = "http://members.unine.ch/jacques.savoy/clef/finnishStemmer.txt"
# stemmer_path = "data/finnishStemmer.txt"
# download(stemmer_url, stemmer_path)

# FinnWordNet
finnwordnet_url = (
    "https://www.kielipankki.fi/download/FinnWordNet/v2.0/FinnWordNet-2.0.zip"
)
finnwordnet_zip_path = "FinnWordNet-2.0.zip"
finnwordnet_extract_path = "data/FinnWordNet"
download_and_extract(finnwordnet_url, finnwordnet_zip_path, finnwordnet_extract_path)

In [3]:
!python -m spacy download fi_core_news_md
!python -m spacy download fi_core_news_sm

### Task 1:

Consider the wordings: “climate change”, “emission”, “resilience”, “sustainability” (need to consider their Finnish translations). Suggest a script where your input each of these wordings and output the corresponding Wikipedia pages, highlighting all the (linked) entities in these pages.


In [6]:
terms = {"ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"}

In [7]:
# Function to load the extracted Wikipedia dataset
def load_wikipedia_dataset(directory):
    data = {}
    data_titles = {}

    # Iterate over all dataset parts
    files = [
        f
        for f in os.listdir(directory)
        if f.startswith("wiki_part") and f.endswith(".VRT")
    ]
    for filename in tqdm(files, desc="Processing files"):
        if filename.startswith("wiki_part") and filename.endswith(".VRT"):
            file_path = os.path.join(directory, filename)
            # print(f"Processing file: {file_path}")

            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

                # Use regex to find all documents within the <doc>...</doc> tags
                docs = re.findall(
                    r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>',
                    content,
                    re.DOTALL,
                )

                # Process each document found
                for doc_id, url, title, doc_content in docs:
                    data_titles[title.casefold()] = {
                        "id": doc_id,
                    }
                    # Only store the data if the title matches one of the specified terms
                    if title.casefold() in terms:
                        data[title.casefold()] = {
                            "id": doc_id,
                            "url": url,
                            "content": doc_content,
                        }
    return data, data_titles


# Load Finnish stopwords
def load_stopwords(file_path):
    with open(file_path, "r", encoding="ISO-8859-1") as f:
        stopwords = f.read().splitlines()
        print(f"Loaded {len(stopwords)} stopwords from {file_path}")
    return stopwords


# Function to highlight linked entities
def highlight_linked_entities(content):
    highlighted_content = re.sub(
        r'(<link entity=")(.*?)(">)(.*?)(</link>)', r"\1\2\3**\4**\5", content
    )
    return highlighted_content


# Function to tokenize text into sentences and remove stopwords
def remove_stopwords(sentence, stopwords, language="finnish"):
    words = word_tokenize(text=sentence, language=language)
    filtered_words = [word for word in words if word.lower() not in stopwords]
    return " ".join(filtered_words)


# Function to remove '#' from the middle of a word
def remove_hash_from_words(text):
    return re.sub(r"(\w)#(\w)", r"\1\2", text)

Load the wikipedia dataset


In [ ]:
dataset_directory = "data/wikipedia-fi-2017-src"  # Path to the extracted dataset
wikipedia_data, wikipedia_title_data = load_wikipedia_dataset(dataset_directory)
print(f"number of wikipedia titles", len(wikipedia_title_data))

Load the finnish stopwords


In [ ]:
finnish_stopwords = load_stopwords(stopwords_path)

Highlight the entities in the wikipedia pages


In [10]:
for title, info in wikipedia_data.items():
    highlighted_content = highlight_linked_entities(info["content"])
    # print(f"Title: {title}\nHighlighted Content: {highlighted_content}\n")

Get the pages for the given wordings and get the sentences


In [ ]:
# Function to extract third column data from sentences
def extract_third_column(sentence):
    third_column_data = []
    lines = sentence.strip().split("\n")
    for line in lines:
        if line.strip() and not line.startswith("<"):
            columns = line.split("\t")
            if len(columns) > 2:
                third_column_data.append(columns[2])
    return third_column_data


def process_paragraphs(input_text):
    # print(f"Input Text: {input_text}")
    paragraphs = re.findall(r"<paragraph>(.*?)</paragraph>", input_text, re.DOTALL)
    all_third_column_data = []
    # print(f"Paragraphs: {paragraphs}")
    for paragraph in paragraphs:
        sentences = re.findall(r"<sentence>(.*?)</sentence>", paragraph, re.DOTALL)
        # print(f"Sentences: {sentences}")
        for sentence in sentences:
            # print(f"Sentence: {sentence}")
            third_column_data = extract_third_column(sentence)
            all_third_column_data.append(" ".join(third_column_data))
    return all_third_column_data


# print(f"Wikipedia data: {wikipedia_data['ilmastonmuutos']['content']}")

# Process each item in wikipedia_data
for item in wikipedia_data.items():
    third_column_data = process_paragraphs(item[1]["content"])
    # print(f"Item {item[0]} third column data:")
    for j, data in enumerate(third_column_data):
        filtered_sentence = remove_stopwords(data, finnish_stopwords)
        # print(f"Sentence {j+1}: {filtered_sentence}")
        # Save the filtered sentence to a variable if needed
        # For example: sentence_var = filtered_sentence
    # print("\n" + "#" * 80 + "\n")


all_words = {}

print(wikipedia_data)
for title, document in wikipedia_data.items():
    sentences = re.findall(
        r"<sentence>(.*?)</sentence>", document["content"], re.DOTALL
    )
    document_words = []
    if sentences:
        for idx, sentence_content in enumerate(sentences):
            # print(f"Processing sentence {idx + 1}...")

            # Split each sentence into individual lines
            lines = sentence_content.strip().split("\n")

            # cleaned_lines = re.sub(r'[^\w\s]', '', lines)
            # Extract the word (3nd column) from each line
            words = [line.split("\t")[2] for line in lines if line.strip()]
            for word in words:
                cleaned_word = re.sub(r"[^\w\s]", "", word)
                if not cleaned_word or cleaned_word in finnish_stopwords:
                    continue
                document_words.append(cleaned_word.casefold())

    all_words[title] = document_words

for title, words in all_words.items():
    print(f"\nWords for document '{title}': {words}")

In [ ]:
entities_found = {}

for i, topic_words in enumerate(all_words):
    found_words = []
    for word in all_words[topic_words]:
        if word.casefold() in [title.casefold() for title in wikipedia_title_data]:
            found_words.append(word.casefold())
    entities_found[topic_words] = found_words


# Example usage of the entities_found dictionary
for topic, found_entities in entities_found.items():
    print(f"Document '{topic}' contains the following entities: {found_entities}")

### Task 2

Assume the content of each webpage is a single document. Use relevant NLTK script to create a corpus constituted of the four document, and appropriate proprocessing and lemmatization, to construct the TfIdfVectorizer of each document and then calculate the cosine similarity of each pair of these documents. Provide the result in a table and comment on the findings.


In [13]:
def extract_second_column(sentence):
    second_column_data = []
    lines = sentence.strip().split("\n")
    for line in lines:
        if line.strip() and not line.startswith("<"):
            columns = line.split("\t")
            if len(columns) > 2:
                second_column_data.append(columns[1])
    return second_column_data


def process_sentences(input_text):
    paragraphs = re.findall(r"<paragraph>(.*?)</paragraph>", input_text, re.DOTALL)
    all_second_column_data = []
    for paragraph in paragraphs:
        sentences = re.findall(r"<sentence>(.*?)</sentence>", paragraph, re.DOTALL)
        # print(f"Sentences: {sentences}")
        for sentence in sentences:
            # print(f"Sentence: {sentence}")
            second_column_data = extract_second_column(sentence)
            all_second_column_data.append(" ".join(second_column_data))
    return all_second_column_data


# Preprocessing function to remove stopwords, stemming, and tokenize the document
def preProcess(doc, stopwords):
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")
    sentences = process_sentences(doc)
    print(f"Sentences: {sentences}")
    # sentences = sent_tokenize(text=sentences.casefold(), language='finnish')

    tokens = []
    for sentence in sentences:
        words = word_tokenize(sentence)

        words = [stemmer.stem(word) for word in words]
        words = [
            word for word in words if word.isalpha() and word not in clean_stopwords
        ]
        words_set = list(set(words))

        tokens.extend(words_set)

    return " ".join(tokens)

In [ ]:
# Preprocess the document
corpus = []

for title, document in wikipedia_data.items():
    processed_doc = preProcess(document["content"], finnish_stopwords)
    print(f"Processed Document: {processed_doc}")
    corpus.append(processed_doc)

# Print the processed document
print("\nCorpus:")
for doc in corpus:
    print(doc)



In [15]:
# TF-IDF Vectorization of each document
tf = TfidfVectorizer(use_idf=True, min_df=1)
tfidf_matrix = tf.fit_transform(corpus)

In [16]:
# Compute cosine similarity of each pair of these documents
cosine_sim = cosine_similarity(tfidf_matrix)

In [ ]:
# Provide the result in a table
cosine_sim_df = pd.DataFrame(
    cosine_sim,
    index=[f"Doc{i+1}" for i in range(len(cosine_sim))],
    columns=[f"Doc{i+1}" for i in range(len(cosine_sim))],
)

print(cosine_sim_df)

# Comment on the findings

### Task 3

Repeat 2) when the documents are restricted only to clickable entities of each document


In [18]:
# Preprocessing function to remove stopwords, stemming
def preProcess_entites(words, stopwords):
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")

    words = [stemmer.stem(word) for word in words]

    words = [word for word in words if word.isalpha() and word not in clean_stopwords]

    words = list(set(words))

    return " ".join(words)

In [ ]:
# Preprocess the document
corpus_entity = []
for document, found_entities in entities_found.items():
    processed_doc = preProcess_entites(found_entities, finnish_stopwords)
    corpus_entity.append(processed_doc)

# Print the processed document
print("\nCorpus_entity:")
for doc in corpus_entity:
    print(doc)

In [20]:
# TF-IDF Vectorization of each document
tf = TfidfVectorizer(use_idf=True, min_df=1)
tfidf_matrix_entity = tf.fit_transform(corpus_entity)

In [21]:
# Compute cosine similarity of each pair of these documents
cosine_sim_entity = cosine_similarity(tfidf_matrix_entity)

In [ ]:
# Provide the result in a table
cosine_sim_df_entity = pd.DataFrame(
    cosine_sim_entity,
    index=[f"Doc{i+1}" for i in range(len(cosine_sim_entity))],
    columns=[f"Doc{i+1}" for i in range(len(cosine_sim_entity))],
)

print(cosine_sim_df_entity)

# Comment on the findings


### Task 4

Write a script that explores the clickable entities in each of the four concepts (exploring only once), and generates new extended vocabulary for each concepts, and compute the new similarity between each pair of concepts using extent of overlapping of overall vocabulary and reduced vocabulary (when restricting the vocabulary to clickable entities). We shall refer to the case where the clickable entities are further explored as extended vocabulary case.


In [61]:


# Function to further explore clickable entities and generate extended vocabulary
# Function to load the extracted Wikipedia dataset
def load_wikipedia_dataset_extended(directory):
    data_entity_voc = {}
    nodup = {}
    categories = ["ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"]
    data_entity_voc = {category: [] for category in categories}
    for topic, found_entities in entities_found.items():
        nodup[topic] = list(set(found_entities))

    # Iterate over all dataset parts
    files = [
        f
        for f in os.listdir(directory)
        if f.startswith("wiki_part") and f.endswith(".VRT")
    ]

    for filename in tqdm(files, desc="Processing files"):
        if filename.startswith("wiki_part") and filename.endswith(".VRT"):
            file_path = os.path.join(directory, filename)
            # print(f"Processing file: {file_path}")

            with open(file_path, "r", encoding="utf-8") as file:
                content = file.read()

                # Use regex to find all documents within the <doc>...</doc> tags
                docs = re.findall(
                    r'<doc id="(.*?)" url="(.*?)" title="(.*?)">(.*?)</doc>',
                    content,
                    re.DOTALL,
                )

                # Process each document found
                for doc_id, url, title, doc_content in docs:
                    # Loop through the relevant categories in 'nodup' to reduce repetitive code
                    for category in categories:
                        # Check if the title matches the category terms in 'nodup'
                        if title.casefold() in nodup[category]:
                            # Create a dictionary with the document data
                            document_data = {
                                "title": title,
                                "id": doc_id,
                                "url": url,
                                "content": doc_content,
                            }

                            # Append the document data to the corresponding category in 'data_entity_voc'
                            data_entity_voc[category].append(document_data)

    return data_entity_voc


In [ ]:
#
dataset_directory = "data/wikipedia-fi-2017-src"  # Path to the extracted dataset
documents_extended = load_wikipedia_dataset_extended(dataset_directory)

In [82]:
def list_extended_words (documents_extended):
    all_words_extended = {}

    for title, documents in documents_extended.items():
        for doc in documents:
            sentences = re.findall(
                r"<sentence>(.*?)</sentence>", doc["content"], re.DOTALL
            )
            document_words = []
            if sentences:
                for idx, sentence_content in enumerate(sentences):
                    # print(f"Processing sentence {idx + 1}...")

                    # Split each sentence into individual lines
                    lines = sentence_content.strip().split("\n")

                    # cleaned_lines = re.sub(r'[^\w\s]', '', lines)
                    # Extract the word (3nd column) from each line
                    words = [line.split("\t")[2] for line in lines if line.strip()]
                    for word in words:
                        cleaned_word = re.sub(r"[^\w\s]", "", word)
                        if not cleaned_word or cleaned_word in finnish_stopwords:
                            continue
                        document_words.append(cleaned_word.casefold())

            if title in all_words_extended:
                all_words_extended[title].extend(document_words)
            else:
                # If the title doesn't exist, initialize it with the document_words list
                all_words_extended[title] = document_words

    return all_words_extended


In [101]:
words_extended = list_extended_words(documents_extended)

words_extended_all = {} # extended words for all documents "ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"

categories = ["ilmastonmuutos", "päästö", "joustavuus", "ekologinen kestävyys"]
words_extended_all = {category: [] for category in categories}

for title, words in all_words.items():
    words_extended_all[title] = all_words[title] + words_extended[title]



In [ ]:
# Preprocessing function to remove stopwords, stemming, and tokenize the document
def preProcess(doc, stopwords):
    clean_stopwords = [word.casefold() for word in stopwords]
    stemmer = SnowballStemmer("finnish")
    sentences = process_sentences(doc)
    print(f"Sentences: {sentences}")
    # sentences = sent_tokenize(text=sentences.casefold(), language='finnish')

    tokens = []
    for sentence in sentences:
        words = word_tokenize(sentence)

        words = [stemmer.stem(word) for word in words]
        words = [
            word for word in words if word.isalpha() and word not in clean_stopwords
        ]
        words_set = list(set(words))

        tokens.extend(words_set)

    return " ".join(tokens)

# Compute similiarity for the extended words
def compute_similarity(vocab1, vocab2):


In [ ]:
compute_similarity(words_extended, words_extended_all)

### Task 5

We want to assess the importance of each of the four concepts by the number of clickable entities including in the webpage of the given concept. Write a script that implements this strategy to determine the importance of each concept. Next, we want to use the extended vocabulary by quantifying the importance of individual entity in the original webpage by the number of clickable links it generates. Summarize a table highlighting the order of importance of the four concepts according to each of the original and extended vocabulary.


In [ ]:
# calculate the occurance of each entity in the entities_found
entity_occurence = {}
for topic, found_entities in entities_found.items():
    entity_occurence[topic] = {}
    for entity in found_entities:
        if entity in entity_occurence[topic]:
            entity_occurence[topic][entity] += 1
        else:
            entity_occurence[topic][entity] = 1

# order the entities by occurance
sorted_entities = {}
for topic, entities in entity_occurence.items():
    sorted_entities[topic] = {
        k: v for k, v in sorted(entities.items(), key=lambda item: item[1], reverse=True)
    }

## print the sorted entities for each topic
for topic, entities in sorted_entities.items():
    print(f"Document '{topic}' contains entities:")
    print(entities)



In [ ]:
## words_extended_all contains the extended vocabulary for each document
## entities_found contains the entities found in each document


def count_clickable_links(entities):
    return {entity: len(entity) for entity in entities}

# Count clickable links for original and extended vocabulary
original_counts = {concept: count_clickable_links(entities) for concept, entities in entities_found.items()}
extended_counts = {concept: count_clickable_links(entities) for concept, entities in words_extended_all.items()}

# Summarize importance
original_importance = {concept: sum(counts.values()) for concept, counts in original_counts.items()}
extended_importance = {concept: sum(counts.values()) for concept, counts in extended_counts.items()}

# Create a DataFrame to summarize the importance
importance_df = pd.DataFrame({
    'Concept': list(original_importance.keys()),
    'Original Importance': list(original_importance.values()),
    'Extended Importance': list(extended_importance.values())
})

# Sort the DataFrame by importance
importance_df = importance_df.sort_values(by=['Original Importance', 'Extended Importance'], ascending=False)

# Display the table
print(importance_df)

### Task 6

We want to assess the similarity between the concepts is reflected in the their webpage content. Use a script to calculate Wu and Palmer WordNet semantic similarity between each pair of the four concepts and then compare result with the Jaccard similarity obtained by both original vocabulary in 2) and extended vocabulary 4), and comment on the compatibility between the semantic similarity and the above constructed Jaccard similarity measures


In [ ]:
finnwordnet_path = "data/FinnWordNet/dict"
wn = WordNetCorpusReader(finnwordnet_path, None)

In [ ]:
# Wu and Palmer WordNet Semantic Similarity Calculation
terms_list = list(terms)
wu_palmer_similarity = []
for i in range(len(terms_list)):
    row = []
    for j in range(len(terms_list)):
        if i == j:
            row.append(1.0)
        else:
            synsets_1 = wn.synsets(terms_list[i], lang='fin')
            synsets_2 = wn.synsets(terms_list[j], lang='fin')
            max_similarity = 0
            for syn1 in synsets_1:
                for syn2 in synsets_2:
                    similarity = syn1.wup_similarity(syn2)
                    if similarity and similarity > max_similarity:
                        max_similarity = similarity
            row.append(max_similarity if max_similarity else 0)
    wu_palmer_similarity.append(row)

# Provide the result in a table
wu_palmer_sim_df = pd.DataFrame(
    wu_palmer_similarity,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

print("\nWu and Palmer WordNet Semantic Similarity:")
print(wu_palmer_sim_df)

# Jaccard Similarity Calculation for Original and Extended Vocabulary
def jaccard_similarity(set1, set2):
    intersection = len(set1.intersection(set2))
    union = len(set1.union(set2))
    return intersection / union if union != 0 else 0

# Calculate Jaccard Similarity for Original and Extended Vocabularies
jaccard_original = []
jaccard_extended = []
for i in range(len(terms_list)):
    row_original = []
    row_extended = []
    for j in range(len(terms_list)):
        if i == j:
            row_original.append(1.0)
            row_extended.append(1.0)
        else:
            row_original.append(jaccard_similarity(all_words[terms_list[i]], all_words[terms_list[j]]))
            row_extended.append(jaccard_similarity(words_extended_allombined_vocab[terwords_extended_allombined_vocab[tercab <- task 4
    jaccard_original.append(row_original)
    jaccard_extended.append(row_extended)

# Provide the results in tables
jaccard_original_df = pd.DataFrame(
    jaccard_original,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

jaccard_extended_df = pd.DataFrame(
    jaccard_extended,
    index=[f"Concept{i+1}" for i in range(len(terms_list))],
    columns=[f"Concept{i+1}" for i in range(len(terms_list))],
)

print("\nJaccard Similarity (Original Vocabulary):")
print(jaccard_original_df)
print("\nJaccard Similarity (Extended Vocabulary):")
print(jaccard_extended_df)

# Comment on Compatibility Between Similarity Measures
print("\nComments on Compatibility Between Semantic and Jaccard Similarities:")
for i in range(len(terms_list)):
    for j in range(i + 1, len(terms_list)):
        print(f"\nComparison between Concept{i+1} and Concept{j+1}:")
        print(f"Wu and Palmer Semantic Similarity: {wu_palmer_sim_df.iloc[i, j]:.2f}")
        print(f"Jaccard Similarity (Original Vocabulary): {jaccard_original_df.iloc[i, j]:.2f}")
        print(f"Jaccard Similarity (Extended Vocabulary): {jaccard_extended_df.iloc[i, j]:.2f}")
        if wu_palmer_sim_df.iloc[i, j] > 0.5:
            print("High semantic similarity is generally compatible with high Jaccard similarity.")
        else:
            print("Low semantic similarity indicates less compatibility with Jaccard similarity.")

### Task 7

Now we want to evaluate the emotion of the text in each dataframe. For this purpose, use WordnetAffect (https://github.com/clemtoy/WNAffect), which generates an emotion state and the part of speech tag of each word matched in the lexicon, and then compile the overall output for each concept. Repeat this process for both restricted and extended vocabulary cases, and comment on the compatibility between the emotion gained from the corresponding Wikipedia page and the intuitive emotion from individual concept.


In [ ]:
# Emotion Analysis Using WordNetAffect
wna = WNAffect('wordnet-1.6/', 'wn-domains-3.2/') # NANNE ÄÄ PITÄÄ LATAAJ

In [ ]:
# Function to analyze emotions of text using WordNetAffect
def analyze_emotions(text):
    words = text.split()
    emotion_count = defaultdict(int)
    for word in words:
        # Translate the Finnish word to English
        translated_word = translator.translate(word, src='fi', dest='en').text
        synsets = wn.synsets(translated_word)
        for syn in synsets:
            emotion = wna.get_emotion(syn)
            if emotion:
                emotion_count[emotion] += 1
    return emotion_count

# Analyze emotions for original and extended vocabulary
emotion_results_original = {}
emotion_results_extended = {}
for term, vocab in all_words.items():
    emotion_results_original[term] = analyze_emotions(" ".join(vocab))
for term, vocab in combined_vocab.items():
    emotion_results_extended[term] = analyze_emotions(" ".join(vocab))

# Print emotion analysis results
print("\nEmotion Analysis (Original Vocabulary):")
for term, emotions in emotion_results_original.items():
    print(f"{term}: {dict(emotions)}")

print("\nEmotion Analysis (Extended Vocabulary):")
for term, emotions in emotion_results_extended.items():
    print(f"{term}: {dict(emotions)}")

# Comment on Compatibility Between Emotion Analysis and Intuitive Emotion
print("\nComments on Compatibility Between Emotion Analysis and Intuitive Emotion:")
for term in terms_list:
    print(f"\nConcept: {term}")
    print(f"Emotion Analysis (Original Vocabulary): {emotion_results_original[term]}")
    print(f"Emotion Analysis (Extended Vocabulary): {emotion_results_extended[term]}")
    # Add your own intuitive analysis based on the concept meaning
    print("The intuitive emotion for this concept aligns/does not align well with the emotion extracted from the Wikipedia page.")words_extended_all

### Task 8

We want to repeat this process for sentiment polarity. For this purpose, use the Vader sentiment https://github.com/cjhutto/vaderSentiment to assess the sentiment of each the Wikipedia page associated to each concept (both with original vocabulary and extended vocabulary, by aggregating the sentiment of individual clickable entity’s page) to compute the sentiment associated with each concept. Comment on the sentiment of each concept in restricted and extended vocabulary and its compatibility with the intuitive sentiment gained from the inherent definition of these concepts.


### Task 9

We want to investigate the similarity using the embedding representation. For this purpose, write a program that uses word2vec embedding to evaluate

i) the consistency of the Wikipedia page by calculating the cosine similarity between the underlined concept and the average word2vec embedding of every words contained in the page

ii) the similarity between the various concepts by computing the cosine similarity between the aggregated embedding vector of the corresponding pages. Comment on the compatibility between webpage content and concept and similarity between the various concepts.


### Task 10

Repeat 9) when the extended vocabulary is considered.


### Task 11

Use appropriate literature to comment on the findings. Also, identify any additional input that would allow you to further elucidate any of the preceding, and use appropriate literature of corpus linguistic literature to justify your findings and comment on the obtained results. Finally, comment on the limitations and structural weakness of the data processing pipeline.
